In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, pickle, time, warnings
import numpy as np, pandas as pd, scipy, scipy.stats as stats, tqdm, h5py
from copy import deepcopy as copy

# Plotting modules
import matplotlib, corner
from pylab import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
# Plotting modules
font = {'family' : 'serif', 'weight' : 'normal',
        'size'   : 16}
legend = {'fontsize': 16}
matplotlib.rc('font', **font)
matplotlib.rc('legend', **legend)
plt.rc('text', usetex=True)
plt.rc('axes', labelsize=16)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('legend',fontsize=16)

cm_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

corner_kwargs = {'max_n_ticks':3, 'title_kwargs':{"fontsize": 16}, 'label_kwargs':{'fontsize':16}}

In [3]:
sys.path.extend(['../utilities/', '../models/'])
import samplers, disk_cone_plcut as dcp, plotting, transformations
from transformations import func_inv_jac, func_labels, label_dict
import functions

In [4]:
import disk_halo_mstogap as dh_msto

In [ ]:
pi_mu, pi_err, abs_sin_lat, m_mu, log_pi_err, hz=1., alpha1=-1., alpha2=-1., alpha3=-1.,
                                Mto=4., Mms=8., Mms1=9., Mms2=7., fD=0.5, Mx=10., R0=8.27, degree=21):

# Test Disk Likelihood gradient

In [251]:
size = 1

In [252]:
kwargs = {'hz':1., 'alpha1':-1., 'alpha2':-1., 'alpha3':-1.,
          'Mto':4., 'Mms':8., 'Mms1':9., 'Mms2':7., 
          'fD':0.5, 'Mx':10., 'R0':8.27, 'degree':21}

In [407]:
sample = {}; true_pars={}; latent_pars={}; cmpt=0;
filename = '/data/asfe2/Projects/mwtrace_data/mockmodel/sample.h'
with h5py.File(filename, 'r') as hf:
    subset = (hf['sample']['M'][...]>5)
    subsample  = np.sort(np.random.choice(np.arange(np.sum(subset)), size=size, replace=False))
    for key in hf['sample'].keys():
        sample[key]=hf['sample'][key][...][subset][subsample]
    # Get true parameters
    for key in hf['true_pars'].keys():
        if not key in np.arange(3).astype(str):
            true_pars[key]=hf['true_pars'][key][...]
        else:
            true_pars[int(key)]={}
            for par in hf['true_pars'][key].keys():
                true_pars[int(key)][par]=hf['true_pars'][key][par][...]
for j in range(3): true_pars[j]['w']*=size
    
args = (sample['parallax_obs'], sample['parallax_error'], sample['sinb'], \
    sample['m_obs'], np.log(sample['parallax_error']))
sample

{'M': array([8.12723482]),
 'cmpt': array([1]),
 'l': array([5.86356284]),
 'm': array([21.09319941]),
 'm_err': array([0.47444547]),
 'm_obs': array([20.84397101]),
 'parallax_error': array([0.39568133]),
 'parallax_obs': array([0.28534232]),
 's': array([3.91915933]),
 'sinb': array([0.88310954]),
 'source_id': array([32722])}

In [348]:
dh_msto.log_expmodel_perr(*args, **kwargs), dh_msto.log_expmodel_perr_grad(*args, **kwargs, grad=True)

(array([-4.1757247]), (array([-4.17366804]), array([[ 1.1690662 ],
         [-0.31573264],
         [ 0.28635836],
         [        nan],
         [ 1.46510109],
         [ 0.05013797]])))

In [349]:
kwargs = {'Mms':8., 'Mms1':9., 'Mms2':7., 
          'Mx':10., 'R0':8.27, 'degree':21}
model = lambda x: dh_msto.log_expmodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[0]
grad = lambda x: dh_msto.log_expmodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[1][:,0]

In [353]:
x0 = np.array([1,-0.5,0.5,4,-1,-1.1])
np.vstack(( scipy.optimize.approx_fprime(x0, model, 1e-10), grad(x0) )).T

array([[ 0.30793146,  0.30770291],
       [-0.51718629, -0.51722961],
       [ 0.7872103 ,  0.78721123],
       [ 0.74306783,         nan],
       [ 0.58413718,  0.58426291],
       [ 0.07648993,  0.07648868]])

# Halo

In [354]:
warnings.filterwarnings('ignore', r'divide by zero encountered')

In [408]:
kwargs = {'Mms':8., 'Mms1':9., 'Mms2':7., 
          'Mx':10., 'R0':8.27, 'degree':11}
model = lambda x: dh_msto.log_halomodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[0]
grad = lambda x: dh_msto.log_halomodel_perr_grad(*args, hz=x[0], alpha3=x[1], fD=x[2], alpha1=x[4], alpha2=x[5],
                                                Mto=x[3],  **kwargs, grad=True)[1][:,0]

In [409]:
x0 = np.array([2.,-1.5,0.5,4,-1,-1.1])
np.vstack(( scipy.optimize.approx_fprime(x0, model, 1e-10), grad(x0) )).T

array([[-1.70722991, -1.7065387 ],
       [ 0.31675107,  0.31731099],
       [-1.97395877, -1.97396354],
       [-0.50544458,         nan],
       [ 0.0074607 ,  0.00745997],
       [ 0.00257572,  0.00256529]])

In [386]:
test_args = (1,3,2.,0.5,0.1,0.,0.,1.)
model = lambda h: dh_msto.halomodel_perr_integrand(0.1, *(1,3,h,0.5,0.1))
grad = lambda h: dh_msto.halomodel_perr_integrand_dh(0.1, *(1,3,h,0.5,0.1))

In [387]:
h = 0.1
np.vstack(( scipy.optimize.approx_fprime(np.array([h]), model, 1e-10), grad(h) )).T

array([[-6.14583813e-07, -6.14583826e-07]])

In [383]:
test_args = (1,3,2.,0.5,0.1,0.,0.,1.)
model = lambda n: dh_msto.halomodel_perr_integrand(0.1, *(1,n,2.,0.5,0.1))
grad = lambda n: dh_msto.halomodel_perr_integrand_dn(0.1, *(1,n,2.,0.5,0.1))

In [388]:
n=0.1
np.vstack(( scipy.optimize.approx_fprime(np.array([n]), model, 1e-10), grad(n) )).T

array([[-6.14583813e-07, -6.14583826e-07]])

In [202]:
test_args = (0.11459087188687923, -1.9315154043531053, 62.460138858539736,
             -0.6133993246056172,  1.002678948333817, 0.2, 0.5709469567273955)
beta, n, h, mu, err, a, b = test_args
grad = lambda x: dh_msto.halomodel_perr_logit_grad(x, test_args) \
               * dh_msto.halomodel_perr_integrand(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand(x, *test_args[:-2])

In [176]:
model(np.array([0.4, 0.5, 0.3]))

array([0.01025454, 0.00886006, 0.00260297])

In [203]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-10)[0], grad(p)

(0.052879228773505815, 0.05287882306911542)

In [220]:
grad = lambda x: dh_msto.halomodel_perr_logit_grad_dn(x, test_args) \
               * dh_msto.halomodel_perr_integrand_dn(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand_dn(x, *test_args[:-2])

In [216]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-10)[0], grad(p)

(-0.02281636685141919, -0.022816013578456487)

In [233]:
grad = lambda x: dh_msto.halomodel_perr_logit_grad_dh(x, test_args) \
               * dh_msto.halomodel_perr_integrand_dh(x, *test_args[:-2])/(x*(beta**2+x**2))
model = lambda x: (x-test_args[-2])*(test_args[-1]-x)*dh_msto.halomodel_perr_integrand_dh(x, *test_args[:-2])

In [237]:
p=0.4
scipy.optimize.approx_fprime(np.array([p]), model, 1e-9)[0], grad(p)

(-0.00014103079598246082, -0.00014103021325713786)